## Task Generator

In this notebook, we will make changes to support a task generator for user simulation study in Tool Sanbbox. The user simulation study has been broken down into two components:
- Natural Language Task Generator: We will take the current randomized state of the environment, all the available tools and prepare a prompt for the model to generate a natural language task.
- User Trajectory Generator: This class will take in a natural language task and try to complete the task in the tool sandbox. Most importantly, we will strip the "thinking" outputs from the outputs of this class. So ideal trajectory should look like a sequence of tool calls and system messages to user.


Additionally, we want to evaluate the *feasibility* of the task generator and *accuracy* of the trajectory generator. Feasbility can be calculated as *how many models are able to complete the task* and accuracy of trajectory generator, we should be able to judge by using milestones and minefields. Milestones and minefields should also be automatically generated but then we also have to find a way to judge it's own accuracy. I know LLM as a judge should be doable but still.

In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
from tool_sandbox.common.execution_context import DatabaseNamespace, RoleType, get_current_context

from tool_sandbox.common.message_conversion import Message
from tool_sandbox.common.tool_conversion import convert_to_openai_tool, convert_to_openai_tools, convert_python_function_to_openai_function


In [ ]:
current_context = get_current_context()

sandbox_database = current_context.get_database(DatabaseNamespace.SANDBOX, get_all_history_snapshots=True, drop_sandbox_message_index=False)


In [ ]:
print(sandbox_database)


In [ ]:
available_tools = current_context.get_available_tools(scrambling_allowed=True)
print(available_tools)


In [ ]:
convert_to_openai_tool(available_tools["add_contact"])


In [ ]:
convert_python_function_to_openai_function("add_contact", available_tools["add_contact"])


In [ ]:
# get all tools and their openai tool representation
from tool_sandbox.common.tool_discovery import get_all_tools, ToolBackend
from tool_sandbox.common.tool_conversion import convert_python_function_to_nl_tool_string, get_tool_docs_natural_language

# TODO: Write a utility function to convert openai_tools to natural language strings for use with LLM.

all_tools = get_all_tools(ToolBackend.DEFAULT)

tool_docs = get_tool_docs_natural_language(all_tools)
print(tool_docs)


